In [24]:
import os
import json
import pandas as pd
import mlflow
from mlflow import log_params, log_metrics
from typing import List, Dict, TypedDict

# LangChain & AI Imports
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableConfig
from langchain_core.messages import HumanMessage, SystemMessage

from typing import List, Literal, TypedDict

# LangGraph Imports
from langgraph.graph import START, END, StateGraph
from langgraph.graph.message import MessagesState

# Ragas Imports
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)
from datasets import Dataset

# Ragas Generation Imports (Necessary for Step 3)
from ragas.testset import TestsetGenerator
from ragas.testset.synthesizers import (
    SingleHopSpecificQuerySynthesizer,
    MultiHopAbstractQuerySynthesizer,
    MultiHopSpecificQuerySynthesizer,
)
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

In [6]:
from dotenv import load_dotenv
load_dotenv()

False

In [ ]:
# ==========================================
# Configuration
# ==========================================
PDF_PATH = "xx.pdf"
CHROMA_PERSIST_DIR = "./chroma_db"
COLLECTION_NAME = "dspy_book_collection"

In [10]:
# Initialize Global Models
llm_engine = ChatOpenAI(model="gpt-4o-mini")
embedding_engine = OpenAIEmbeddings( model="text-embedding-3-large")

lm_judge = ChatOpenAI(
    model="gpt-4o",
    temperature=0.0,
)

### 1.Vectorizing the Knowledge Base

In [11]:
# ==========================================
# Step 1 & 2: Extract & Vectorize (Chroma)
# ==========================================
def process_and_vectorize_pdf(file_path: str, persist_dir: str, collection_name: str = COLLECTION_NAME):
    """Loads PDF, splits, and saves to ChromaDB."""
    print(f"--- Loading {file_path} ---")
    loader = PyPDFLoader(file_path)
    documents = loader.load() # Defaults to page-by-page chunking

    print(f"--- Vectorizing {len(documents)} pages to ChromaDB ---")
    vectorstore = Chroma.from_documents(
        documents=documents,
        embedding=embedding_engine,
        collection_name = collection_name,
        persist_directory=persist_dir
    )
    return vectorstore, documents

In [12]:
chromaVectorStore, langchainDocLs = process_and_vectorize_pdf("complete-book.pdf", "./chroma_db")

--- Loading complete-book.pdf ---


Got invalid hex string: Odd-length string (b'1f5a5')
Got invalid hex string: Odd-length string (b'1f4e6')
Got invalid hex string: Odd-length string (b'1f517')
Got invalid hex string: Odd-length string (b'1f4da')
Got invalid hex string: Odd-length string (b'1f680')
Got invalid hex string: Odd-length string (b'1f3a7')
Got invalid hex string: Odd-length string (b'1f5a5')
Got invalid hex string: Odd-length string (b'1f4e6')
Got invalid hex string: Odd-length string (b'1f517')
Got invalid hex string: Odd-length string (b'1f4da')
Got invalid hex string: Odd-length string (b'1f680')
Got invalid hex string: Odd-length string (b'1f3a7')
Got invalid hex string: Odd-length string (b'1f5a5')
Got invalid hex string: Odd-length string (b'1f4e6')
Got invalid hex string: Odd-length string (b'1f517')
Got invalid hex string: Odd-length string (b'1f4da')
Got invalid hex string: Odd-length string (b'1f680')
Got invalid hex string: Odd-length string (b'1f3a7')
Got invalid hex string: Odd-length string (b'1

--- Vectorizing 254 pages to ChromaDB ---


In [13]:
len(langchainDocLs)

254

In [14]:
langchainDocLs[108]

Document(metadata={'producer': 'Asciidoctor PDF 2.3.20, based on Prawn 2.4.0', 'creator': '', 'creationdate': '2025-12-14T17:21:32+05:00', 'title': 'Untitled', 'moddate': '2025-12-14T17:21:29+05:00', 'source': 'complete-book.pdf', 'total_pages': 254, 'page': 108, 'page_label': '108'}, page_content='Integrating DSPy with MCP Server\nPlaywright is an open-source automation framework created by Microsoft for\nprogrammatic browser control. It allows you to automate actions in modern\nbrowsers like Chromium (Chrome, Edge), Firefox, and WebKit (Safari) across\nWindows, macOS, and Linux. The playwright-mcp package is an MCP server that\nexposes tools for browser control.\nPlaywright MCP Repository - https://github.com/microsoft/playwright-mcp\nLet us run the Playwright MCP Server.\nInstalling playwright MCP Server\n$ npx @playwright/mcp@latest --port 8931\nListening on http://localhost:8931\nPut this in your client config:\n{\n\xa0 "mcpServers": {\n\xa0   "playwright": {\n\xa0     "url": "htt

In [ ]:
### 2. Generating Synthetic Dataset

In [18]:
# ==========================================
# Step 3: Generate Synthetic Test Sets
# ==========================================
def save_for_dspy(testset_df: pd.DataFrame, filename: str):
    """Saves Ragas testset in DSPy-compatible JSON format."""
    dspy_data = []
    for _, row in testset_df.iterrows():
        entry = {
            "question": row['user_input'],
            "answer": row['reference'],
            "gold_context": row['reference_contexts'],
            "metadata": {"synthesizer": row.get('synthesizer_name', 'unknown')}
        }
        dspy_data.append(entry)

    with open(filename, 'w') as f:
        json.dump(dspy_data, f, indent=4)
    print(f"Saved DSPy dataset: {filename}")

def generate_ragas_testsets(documents, num_of_questions = 100):
    """Generates Single, Multi-hop, and Mixed test sets."""
    print("--- Initializing Ragas Testset Generator ---")

    # Wrappers for Ragas
    generator_llm = LangchainLLMWrapper(llm_engine)
    generator_embeddings = LangchainEmbeddingsWrapper(embedding_engine)

    generator = TestsetGenerator(
        llm=generator_llm,
        embedding_model=generator_embeddings
    )

    # Define Synthesizer Distributions
    distributions = {
        "single_hop": [
            (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 1.0)
        ],
    }

    '''
        "multi_hop": [
            (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.5),
            (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.5)
        ],
        "mixed": [
            (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
            (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
            (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25)
        ]
    }
    '''

    test_size = num_of_questions  # Small size for demo; increase for production


    datasetLs = []
    dataFrameLs = []
    for name, dist in distributions.items():
        print(f"Generating {name} testset...")
        testset = generator.generate_with_langchain_docs(
            documents,
            testset_size=test_size,
            query_distribution=dist
        )

        df = testset.to_pandas()
        save_for_dspy(df, f"ragas_testset_{name}.json")
        datasetLs.append(testset)
        dataFrameLs.append(df)

    return datasetLs, dataFrameLs

In [19]:
#Generate single dataset
synthethicDatasetLs, datasetDFLs = generate_ragas_testsets(langchainDocLs)

C:\Users\marci\AppData\Local\Temp\ipykernel_55856\1298520239.py:25: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  generator_llm = LangchainLLMWrapper(llm_engine)
C:\Users\marci\AppData\Local\Temp\ipykernel_55856\1298520239.py:26: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  generator_embeddings = LangchainEmbeddingsWrapper(embedding_engine)


--- Initializing Ragas Testset Generator ---
Generating single_hop testset...


Applying CustomNodeFilter:   0%|          | 0/254 [00:00<?, ?it/s]Node 2f9962cb-f417-4612-ba72-9679f36806f9 does not have a summary. Skipping filtering.
Node 691bd0a0-d788-4aa8-8c21-c61a5f24bf58 does not have a summary. Skipping filtering.
Applying CustomNodeFilter:   1%|          | 2/254 [00:02<06:16,  1.49s/it]Node 66748b17-b084-4733-b2fa-b702dfcafb47 does not have a summary. Skipping filtering.
Node 37ff9cdd-7749-46e1-a09a-12a5b8d1de2f does not have a summary. Skipping filtering.
Applying CustomNodeFilter:  51%|█████     | 129/254 [05:21<04:33,  2.19s/it]Node 08dbe310-17f3-4bee-b1c6-6939f9f63ad7 does not have a summary. Skipping filtering.
Node 43251ee1-d94d-4798-81a5-81e0a9d15b09 does not have a summary. Skipping filtering.
Applying CustomNodeFilter:  85%|████████▍ | 215/254 [09:23<01:27,  2.24s/it]Node f0db820b-4cf3-40bf-b698-0160ffcdecc9 does not have a summary. Skipping filtering.
Node 8c60fdf3-70a7-4cab-899b-73901e3a8d65 does not have a summary. Skipping filtering.
Applying Cus

Saved DSPy dataset: ragas_testset_single_hop.json


In [16]:
synthethicDatasetLs, datasetDFLs = generate_ragas_testsets(langchainDocLs)

--- Initializing Ragas Testset Generator ---
Generating single_hop testset...


C:\Users\marci\AppData\Local\Temp\ipykernel_55856\934105838.py:25: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  generator_llm = LangchainLLMWrapper(llm_engine)
C:\Users\marci\AppData\Local\Temp\ipykernel_55856\934105838.py:26: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  generator_embeddings = LangchainEmbeddingsWrapper(embedding_engine)
Applying CustomNodeFilter:   0%|          | 0/254 [00:00<?, ?it/s]Node acda5c8a-482f-424a-9c3b-3b1de4ca33f4 does not have a summary. Skipping filtering.
Node 75baf941-8f13-4777-a5e3-

Saved DSPy dataset: ragas_testset_single_hop.json
Generating multi_hop testset...


Applying CustomNodeFilter:   0%|          | 0/254 [00:00<?, ?it/s]Node e7851736-b5ab-4dc0-8edd-dda5bf361911 does not have a summary. Skipping filtering.
Node 4ff6d630-d8ee-4642-9975-888fd1979693 does not have a summary. Skipping filtering.
Applying CustomNodeFilter:   1%|          | 2/254 [00:04<08:56,  2.13s/it]Node fbeaddb3-c3ee-4687-96ff-5c088ca571f3 does not have a summary. Skipping filtering.
Node 5e78e839-e1f8-424a-b5c5-f4c345b54c86 does not have a summary. Skipping filtering.
Applying CustomNodeFilter:  51%|█████     | 130/254 [05:08<03:19,  1.61s/it]Node c7d6fd1b-f3c8-496d-bc9e-adc355adbf16 does not have a summary. Skipping filtering.
Node abed6d01-ec15-4cd7-bff3-39562965beee does not have a summary. Skipping filtering.
Applying CustomNodeFilter:  86%|████████▌ | 218/254 [08:54<01:11,  2.00s/it]Node 9aa02081-dfe8-4fe4-b36e-da71901635c7 does not have a summary. Skipping filtering.
Node 7d34c719-d8cb-4cd3-95a3-c9c2664e2135 does not have a summary. Skipping filtering.
Applying Cus

Saved DSPy dataset: ragas_testset_multi_hop.json
Generating mixed testset...


Applying CustomNodeFilter:   0%|          | 0/254 [00:00<?, ?it/s]Node e3bbd6aa-4719-4132-9c58-5562f5c5bab3 does not have a summary. Skipping filtering.
Node 0ae33412-9cd8-4750-8ce1-d103e214b285 does not have a summary. Skipping filtering.
Applying CustomNodeFilter:   1%|          | 2/254 [00:02<06:04,  1.45s/it]Node 1d1356f7-de11-4aff-ada3-f0d3e8a758e8 does not have a summary. Skipping filtering.
Node f7dab48e-a594-4303-8999-69933708062f does not have a summary. Skipping filtering.
Applying CustomNodeFilter:  51%|█████     | 129/254 [04:34<03:20,  1.61s/it]Node f4f4bacf-7cd0-43d7-81e3-36a3a5da47ba does not have a summary. Skipping filtering.
Node 435c91a0-326f-47e6-a2ca-7a723433aa6e does not have a summary. Skipping filtering.
Applying CustomNodeFilter:  85%|████████▌ | 216/254 [07:48<01:06,  1.75s/it]Node ab645e81-ec8d-472e-9c32-145c04192d79 does not have a summary. Skipping filtering.
Node 9b1beaa6-00f1-46f9-b2bc-60c02922b324 does not have a summary. Skipping filtering.
Applying Cus

Saved DSPy dataset: ragas_testset_mixed.json


###3. Defining RAG Agent

In [25]:
class State(TypedDict):
    question: str
    documents: List[Document]
    loop_step: int
    evaluation: str  # "Sufficient" or "Insufficient"
    answer: str

In [26]:
#3. Helper to get LLM from Config

def get_llm(config: RunnableConfig) -> ChatOpenAI:
    """Extracts the LLM from the configuration."""
    configurable = config.get("configurable", {})
    llm = configurable.get("llm")
    if not llm:
        raise ValueError("No LLM instance found in config. Please pass it via 'configurable'.")
    return llm

def get_retriever(config: RunnableConfig):
    """Extracts the retriever from the configuration."""
    configurable = config.get("configurable", {})
    retriever = configurable.get("retriever")
    if not retriever:
        raise ValueError("No 'retriever' found in config.")
    return retriever

In [27]:
# 4. Nodes - RAG Agent nodes
def rewrite_query(state: State, config: RunnableConfig):
    """
    Rewrites the query if retrieval failed.
    """
    llm = get_llm(config) # <--- Get LLM from config

    question = state["question"]
    loop_step = state.get("loop_step", 0)

    if loop_step == 0:
        print(f"---STEP {loop_step}: INITIAL QUERY PASS-THROUGH---")
        return {"loop_step": loop_step + 1}

    print(f"---STEP {loop_step}: REWRITING QUERY---")

    msg = [
        SystemMessage(content="You are a helpful assistant that optimizes queries for vector retrieval."),
        HumanMessage(content=f"Look at the initial question: {question}. Formulate an improved question to find better results.")
    ]
    better_question = llm.invoke(msg).content

    return {"question": better_question, "loop_step": loop_step + 1}

In [28]:
def retrieve(state: State, config: RunnableConfig):
    """
    Retrieve documents using the injected retriever.
    """
    retriever = get_retriever(config)  # <--- Get Retriever from config

    print("---RETRIEVING DOCUMENTS---")
    question = state["question"]

    # Support both raw vector stores and dedicated retrievers
    if hasattr(retriever, "invoke"):
        # It's a standard LangChain Retriever
        retrieved_docs = retriever.invoke(question)
    elif hasattr(retriever, "similarity_search"):
        # It's a VectorStore object (like Chroma)
        retrieved_docs = retriever.similarity_search(question)
    else:
        raise ValueError("Injected object is neither a Retriever nor a VectorStore")

    return {"documents": retrieved_docs}

In [29]:
# 4. Nodes - RAG Agent nodes
def evaluator(state: State, config: RunnableConfig):
    """
    Evaluates if the retrieved documents are sufficient.
    """
    llm = get_llm(config) # <--- Get LLM from config

    print("---EVALUATING DOCUMENTS---")
    question = state["question"]
    documents = state["documents"]
    docs_content = "\n\n".join(doc.page_content for doc in documents)

    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an expert evaluator. Given the context, determine if it is sufficient to answer the user question."),
        ("human", "Question: {question}\n\nContext: {context}\n\nIs the context sufficient? Return only 'YES' or 'NO'.")
    ])

    chain = prompt | llm | StrOutputParser()
    score = chain.invoke({"question": question, "context": docs_content})

    status = "Sufficient" if "YES" in score.upper() else "Insufficient"
    print(f"---EVALUATION: {status}---")
    return {"evaluation": status}

In [30]:
#4. Nodes - RAG Agent nodes

def generate(state: State, config: RunnableConfig):
    """
    Generates the final answer.
    """
    llm = get_llm(config) # <--- Get LLM from config

    print("---GENERATING ANSWER---")
    question = state["question"]
    documents = state["documents"]
    docs_content = "\n\n".join(doc.page_content for doc in documents)

    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant. Use the context to answer the question. If you don't know, say so."),
        ("human", "Question: {question}\n\nContext: {context}\n\nAnswer:")
    ])

    chain = prompt | llm | StrOutputParser()
    answer = chain.invoke({"question": question, "context": docs_content})
    return {"answer": answer}

In [31]:
# 5. Router (Unchanged)
def router(state: State) -> Literal["generate", "rewrite_query"]:
    evaluation = state["evaluation"]
    loop_step = state["loop_step"]
    if evaluation == "Sufficient": return "generate"
    if loop_step <= 3: return "rewrite_query"
    return "generate"

In [32]:
# 6. Graph Construction
workflow = StateGraph(State)

workflow.add_node("rewrite_query", rewrite_query)
workflow.add_node("retrieve", retrieve)
workflow.add_node("evaluator", evaluator)
workflow.add_node("generate", generate)

workflow.add_edge(START, "rewrite_query")
workflow.add_edge("rewrite_query", "retrieve")
workflow.add_edge("retrieve", "evaluator")

workflow.add_conditional_edges(
    "evaluator",
    router,
    {"rewrite_query": "rewrite_query", "generate": "generate"}
)

workflow.add_edge("generate", END)
app = workflow.compile()

In [41]:
llm_engine

ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x00000211C0E60510>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000211C0E60770>, root_client=<openai.OpenAI object at 0x00000211C0E0B460>, root_async_client=<openai.AsyncOpenAI object at 0x00000211C0E0B570>, model_name='gpt-4o-mini', temperature=0.01, model_kwargs={}, openai_api_key=SecretStr('**********'), request_timeout=180, stream_usage=True, n=1)

In [35]:
# 7. Execution with Injected LLM
# B. Prepare Retriever (Setup Chroma)

# Convert vector store to a standard retriever interface
my_retriever = chromaVectorStore.as_retriever(search_kwargs={"k": 3})

# C. Invoke with Injection
initial_input = {"question": "What is the role of signatures in DSPY?", "loop_step": 0}

result = app.invoke(
    initial_input,
    config={
        "configurable": {
            "llm": llm_engine,
            "retriever": my_retriever
        }
    }
)

print("\nFinal Answer:")
print(result["answer"])

---STEP 0: INITIAL QUERY PASS-THROUGH---
---RETRIEVING DOCUMENTS---
---EVALUATING DOCUMENTS---
---EVALUATION: Sufficient---
---GENERATING ANSWER---

Final Answer:
In DSPY, signatures play a crucial role in defining the structure of inputs and outputs for interactions with language models. They provide clear definitions that enhance type safety, automatic validation, and model portability. Signatures allow developers to work independently by introducing new fields as requirements evolve, making them team-friendly and reducing the complexity of managing shared prompt templates. Additionally, signatures enable DSPY to automatically optimize prompts based on the provided data, which is particularly beneficial as applications grow in complexity. Overall, signatures help maintain clarity and organization in the programming process while facilitating effective communication with language models.


In [ ]:
#

### 4. Evaluating the RAG Agent

In [50]:
# ==========================================
# Step 9: Evaluate with Ragas & MLflow
# ==========================================
def evaluate_langgraph_agent(app, testsets_generator, appConfig):
    """Evaluates the LangGraph app using Ragas and logs to MLflow."""

    # Ragas Metrics
    metrics = [faithfulness, answer_relevancy, context_precision, context_recall]

    # Iterate over generated testsets
    for name, test_df in testsets_generator:
        print(f"\n--- Evaluating Testset: {name} ---")

        # Start MLflow Run
        with mlflow.start_run(run_name=f"RAG_Eval_{name}"):
            log_params({"testset_type": name, "model": "gpt-4o", "vector_db": "Chroma"})

            questions = test_df['user_input'].tolist()
            ground_truths = test_df['reference'].tolist()

            answers = []
            contexts = []

            # Run Inference
            for q in questions:
                # Invoke LangGraph
                print(q)
                result = app.invoke({"question": q}, config=appConfig)

                answers.append(result["answer"])
                # Extract page content for Ragas
                retrieved_texts = [doc.page_content for doc in result["documents"]]
                contexts.append(retrieved_texts)

            # Prepare Ragas Dataset
            eval_data = {
                "question": questions,
                "answer": answers,
                "contexts": contexts,
                "ground_truth": ground_truths
            }
            dataset = Dataset.from_dict(eval_data)

            # Run Evaluation
            results = evaluate(
                dataset,
                metrics=metrics,
                llm=LangchainLLMWrapper(llm_engine),
                embeddings=LangchainEmbeddingsWrapper(embedding_engine)
            )

            print(f"Results for {name}: {results}")

            # Log Metrics to MLflow
            for metric_name, score in results.items():
                log_metrics({metric_name: score})

            # Save CSV artifact
            csv_name = f"eval_results_{name}.csv"
            results.to_pandas().to_csv(csv_name, index=False)
            mlflow.log_artifact(csv_name)


In [51]:
 datasetDFLs[0].head()

,user_input,reference_contexts,reference,persona_name,query_style,query_length,synthesizer_name
0,What is the purpose of Ollama in the setup pro...,[Table of Contents\nChapter 1: DSPy - From Pro...,"Ollama is part of the setup process, specifica...",AI Application Developer,PERFECT_GRAMMAR,SHORT,single_hop_specific_query_synthesizer
1,What role do metrics play in evaluating softwa...,[Composition Patterns . . . . . . . . . . . . ...,Metrics are essential in evaluating software a...,Software Architect,WEB_SEARCH_LIKE,MEDIUM,single_hop_specific_query_synthesizer
2,What is DSPy and how it used in MLflow for emb...,[MLflow Setup and Installation . . . . . . . ....,DSPy is mentioned in the context of embeddings...,Software Architect,POOR_GRAMMAR,LONG,single_hop_specific_query_synthesizer
3,What is an Artificial Neural Network and how i...,[General purpose reranker models (small & effi...,An Artificial Neural Network is a computationa...,Software Architect,WEB_SEARCH_LIKE,MEDIUM,single_hop_specific_query_synthesizer
4,How does disinformation impact the development...,[Chatbot . . . . . . . . . . . . . . . . . . ....,Disinformation can significantly hinder the de...,AI Application Developer,PERFECT_GRAMMAR,LONG,single_hop_specific_query_synthesizer


In [ ]:
#Executing RAGAS Evaluation
evalConfig={
        "configurable": {
            "llm": llm_engine,
            "retriever": my_retriever
        }
    }

testsetLs  = [("synthetic", datasetDFLs[0])]
evaluate_langgraph_agent(app, testsetLs, evalConfig)


--- Evaluating Testset: synthetic ---
What is the purpose of Ollama in the setup process?
---STEP 0: INITIAL QUERY PASS-THROUGH---
---RETRIEVING DOCUMENTS---
---EVALUATING DOCUMENTS---
---EVALUATION: Sufficient---
---GENERATING ANSWER---
What role do metrics play in evaluating software architecture?
---STEP 0: INITIAL QUERY PASS-THROUGH---
---RETRIEVING DOCUMENTS---
---EVALUATING DOCUMENTS---
---EVALUATION: Insufficient---
---STEP 1: REWRITING QUERY---
---RETRIEVING DOCUMENTS---
---EVALUATING DOCUMENTS---
---EVALUATION: Insufficient---
---STEP 2: REWRITING QUERY---
---RETRIEVING DOCUMENTS---
---EVALUATING DOCUMENTS---
---EVALUATION: Insufficient---
---STEP 3: REWRITING QUERY---
---RETRIEVING DOCUMENTS---
---EVALUATING DOCUMENTS---
---EVALUATION: Insufficient---
---GENERATING ANSWER---
What is DSPy and how it used in MLflow for embedding and retrieval?
---STEP 0: INITIAL QUERY PASS-THROUGH---
---RETRIEVING DOCUMENTS---
---EVALUATING DOCUMENTS---
---EVALUATION: Sufficient---
---GENERATI

In [ ]:
#